In [1]:
#initial

import pandas as pd
import random
import time
import matplotlib.pyplot as plt
from datetime import datetime

words = pd.read_excel('/Users/aleksejgukov/Desktop/dutch.xlsx',sheet_name='update')
words = words.loc[:,'word':]
lesson = pd.read_excel('/Users/aleksejgukov/Desktop/dutch.xlsx',sheet_name='lesson')
lesson = lesson.loc[:,'lesson':]

In [23]:
import math

In [59]:
#classes

class Words(object):
    def __init__(self, word, typ, translation, russian, example, example_translation, appear, trial_d, trial_r, success, weight):
        self.word = word
        self.typ = typ
        self.translation = translation
        self.example = example
        self.example_translation = example_translation
        self.russian = russian
        self.appear = appear
        self.trial_d = trial_d
        self.trial_r = trial_r
        self.success = success
        self.weight = weight
            
    def getTyp(self):
        return self.typ
    
    def getRussian(self):
        return self.russian
    
    def getExample(self):
        return self.example
    
    def changeType(self,new):
        self.typ = new
    
    def getTrials_d(self):
        return self.trial_d
    
    def getTrials_r(self):
        return self.trial_r
    
    def addTrials_d(self):
        self.trial_d += 1
        
    def addTrials_r(self):
        self.trial_r += 1
    
    def getAppear(self):
        return self.appear
    
    def addAppear(self):
        self.appear += 1
        return self.appear
       
    def addSuccess(self):
        self.success += 1
        return self.success   
    
    def getSuccess(self):
        return self.success
    
    def getWord(self):
        return self.word
    
    def getTranslation(self):
        return self.translation
    
    def getWeight(self):
        if self.trial_d == 0 and self.trial_r == 0:
            return self.weight
        else: 
            return 100 - (self.success/ (self.trial_d + self.trial_r)) * 100
        
    
    def __len__(self):
        return len(f'{self.word} -> {self.translation}')
    
    def __repr__(self):
        return f'{self.word}: {self.getTranslation()} , {self.getExample()}'
    
    def __str__(self):
        return f'{self.word} : {self.translation}'
    
    
class Lesson(object):
    def __init__(self, number):
        self.number = number
    def getNumber(self):
        return self.number
    def start(self, start):
        self.start = start
    def getStart(self):
        return self.start
    def inter(self, inter):
        self.inter = inter
    def getInter(self):
        return self.inter
    def finish(self, finish):
        self.finish = finish
    def getFinish(self):
        return self.finish
    def number_of_easy(self,easy):
        self.easy = easy
    def getNumber_of_easy(self):
        return self.easy
    def points(self, pts):
        self.pts = pts
    def add_pts(self, a_pts):
        self.pts += a_pts
    def getPoints(self):
        return self.pts + 1500 - int((self.finish - self.start).seconds)
    def wlist(self,list_of_words):
        self.list_of_words = list_of_words
    def getList(self):
        return self.list_of_words
    def length_of_lesson(self,lol):
        self.lol = lol
    def getLength_of_lesson(self):
        return self.lol

In [60]:
#defs

def loadWords(df_words, temp):
    wordList = []
    for i in range (len(words)):
        if temp == 'yes':
            wordList.append(Words(words.iloc[i,0],words.iloc[i,1],words.iloc[i,2],words.iloc[i,5], words.iloc[i,3],words.iloc[i,4],words.iloc[i,6],words.iloc[i,7],words.iloc[i,8],words.iloc[i,9],words.iloc[i,10]))
        else:
            wordList.append(Words(words.iloc[i,0],words.iloc[i,1],words.iloc[i,2],words.iloc[i,5], words.iloc[i,3],words.iloc[i,4],0,0,0,0, 100))       
    return wordList


def no_space(word):
    lst = []
    for w in word:
        lst.append(w.strip())
    return lst


def translation_with_comma(translation):
    try:
        translation = translation.split(',')
        translation = no_space(translation)
        return translation
    
    except:
        return [translation]
    
def help_for_guess(word, r): #r - number of letters
    list_of_word = list(word)
    copy = list_of_word.copy()
    for j in range(len(list_of_word)):
        if list_of_word[j] == ' ':
            copy.remove(list_of_word[j])
    
    l = random.sample(copy, k = r)
    
    for i in range(len(list_of_word)):
        if list_of_word[i] in l:
            list_of_word[i] = list_of_word[i]
        else:
            list_of_word[i] = '_'      
    back = ' '.join(list_of_word)
    
    return back


def rightword(word,translation,rever=0):
    point_counter = 0
    if rever == 1:
        b = 0
        while b == 0:
            x = input(f'[à ë ï é]\n\nPress "1","2","3" to open 1, 2, 3 letters in the word\n\n {translation}: ')
            if x == word:
                b = 1
                return [True, point_counter]
            elif x == '1' or x == '2' or x == '3':
                print(help_for_guess(word, int(x)))
                point_counter += int(x)
            else:
                return [False, point_counter]
                b = 1
    elif rever == 0:
        b = 0
        translation = translation_with_comma(translation)
        while b == 0:
            x = input(f'[à ë ï é]\n\nPress "1","2","3" to open 1, 2, 3 letters in the word\n\n {word}: ')
            if x in translation:
                b = 1
                return [True, point_counter]
            elif x == '1' or x == '2' or x == '3':
                print(help_for_guess(translation[0], int(x)))
                point_counter += int(x)
            else:
                b = 1
                return [False, point_counter]
    

def the_longest(sample_of_words):
    mx = 0
    for i in sample_of_words:
        if len(i) > mx:
            mx = len(i)
    return mx


def cycle(sample_of_words, rever):
    p = 250 
    s = sample_of_words.copy()
    while len(s) > 0:
        random.shuffle(s)
        lst_to_delete = []
        for i in s:
            temp = rightword(i.getWord(),i.getTranslation(),rever)
            if temp[0]:
                print("It's right!")
                p = p - temp[1]
                print(temp[1], p)
                i.addSuccess()
                if rever == 0:
                    i.addTrials_d()
                else:
                    i.addTrials_r()
                lst_to_delete.append(i)
            else:
                print("Shame on you!")
                p = p - temp[1]
                p -= 1
                print(p)
                if rever == 0:
                    i.addTrials_d()
                else:
                    i.addTrials_r()            
        if len(lst_to_delete) > 0:
            for w in lst_to_delete:
                s.remove(w)
            if len(s) != 0:
                plotting(s)
        else:
            if len(s) != 0:
                plotting(s)
            
            
    return p

def list_to_list(lst):
    st = ''
    for w in lst:
        st = st + w + '; '
    st.strip()       
    st = st[0:(len(st)-2)]
    return st

def lessonLenght(list_of_words):
    lessonLenght = 0
    for w in list_of_words:
        temp = w.getWord()
        temp = temp.replace(' ','')
        lessonLenght += len(temp)
    return lessonLenght

def random_sample(list_of_words, n):
    sample = random.choices(list_of_words, weights = [w.getWeight() for w in list_of_words], k = n )
    l = len(sample)
    st = 0
    count = 0
    while l != st:
        st = len(set(sample))
        sample = random.choices(list_of_words, weights = [w.getWeight() for w in list_of_words], k = n )
        count += 1
    print(count)
    
    return sample

In [61]:
#object creation
exist = 'yes'
wordList = loadWords(words,exist)
sample = random_sample(wordList,25)   

1


In [62]:
sample

[het zeilen: sailing , nan,
 mineur: minor , nan,
 maarja: but then , Zij had toen een vaste vriend, maarja die kerel was haar niet waard.,
 favoriete: favourite , nan,
 iedere: every , nan,
 eekhoorn: squirrel , nan,
 gegeven: fact, data , Uitgebreid onderzoek bracht geen nieuwe gegevens aan het licht.,
 notulen: minutes of a meeting , nan,
 zanger: singer , Het is een misvatting dat iedere zanger ook componist en tekstdichter is van het door hem vertolkte lied.,
 ideaal: ideal , Het leek haar een ideale omgeving om te werken.,
 voorspellen: to predict , Zij had de overwinning al voorspeld.,
 krankzinnig: insane , We hebben een krankzinnig gesprek gehad.,
 van harte: congratulations , nan,
 kunstenaar: artist , De kamers hangen vol met werk van bevriende kunstenaars.,
 navel: navel , nan,
 gebaren: to gesture , Ze zei niets en gebaarde dat hij haar moest volgen.,
 kortsluiting: shortcircuit , nan,
 opeens: suddenly , Hij kwam opeens binnen.,
 zonder: without , Er werden in enkele geva

In [63]:
print(sample[23])

italië : Italy


In [28]:
x = sample[23].getExample()
#math.isnan(x)

In [29]:
x

'nan : nan'

In [51]:
sample

[hoodfkwartier: headquarters , nan,
 naar verhouding: relatively , nan,
 bevinding: finding , nan,
 gazon: lawn , nan,
 net: net , De dolfijn raakte verstrikt in de netten van de vissers.,
 visum: visa , nan,
 inspiratie: inspiration , Hij mist inspiratie en creativiteit om zelf iets te ontwerpen.,
 middelbare school: secondary school , nan,
 waarna: after which , We kregen nog ijs, waarna we afsloten met koffie.,
 wonen: to live , Bovendien hebben we toffe buren en dat maakt het des te leuker om hier te wonen.,
 uitgaan van: assume , nan,
 ellende: misery , Veel mensen vrezen dat er nog meer ellende in het verschiet ligt.,
 dergelijk: such , Ik heb nooit eerder van dergelijke zaken gehoord.,
 vlotter: float , nan,
 liever: rather , Ik zou liever zelf met hem meegaan.,
 werkdag: working day , De werken zullen 125 werkdagen in beslag nemen.,
 tik: tap , Nu stapte hij onverhoeds binnen na een stevige tik op de deur.,
 ventilator: fan , nan,
 veulen: foal , nan,
 brandstof: fuel , Vooral 

In [64]:
for w in sample:
    try:
        math.isnan(w.getExample())
    except:
        print(w.getExample())

Zij had toen een vaste vriend, maarja die kerel was haar niet waard.
Uitgebreid onderzoek bracht geen nieuwe gegevens aan het licht.
Het is een misvatting dat iedere zanger ook componist en tekstdichter is van het door hem vertolkte lied.
Het leek haar een ideale omgeving om te werken.
Zij had de overwinning al voorspeld.
We hebben een krankzinnig gesprek gehad.
De kamers hangen vol met werk van bevriende kunstenaars.
Ze zei niets en gebaarde dat hij haar moest volgen.
Hij kwam opeens binnen.
Er werden in enkele gevallen etenswaren zonder vergunning verkocht.
Al snel was ze haar aanvankelijke angst kwijt en begon ze zelfs aardigheid in het rijden te krijgen.
De aanvraag voor de vergunning werd afgewezen.


In [76]:
temp = []
for w in sample: 
    try:
        math.isnan(w.getTyp())
        temp.append(w)
    except:
        print(w.getTyp(), w)
for t in temp:
    print(t)

noun het zeilen : sailing
adjective mineur : minor
interj maarja : but then
adjective, noun favoriete : favourite
de eekhoorn : squirrel
noun, het gegeven : fact, data
de notulen : minutes of a meeting
noun, de(m) zanger : singer
adj ideaal : ideal
verb voorspellen : to predict
adj krankzinnig : insane
noun, de(m) kunstenaar : artist
de navel : navel
verb gebaren : to gesture
de kortsluiting : shortcircuit
adv opeens : suddenly
prep zonder : without
de weergave : reproduction
adj aanvankelijk : initial
de boef : crook, bad guy
noun, de aanvraag : application
de zelfbeheersing : self-control
iedere : every
van harte : congratulations
italië : Italy


In [73]:
len(temp)

3

In [74]:
temp

[iedere: every , nan, van harte: congratulations , nan, italië: Italy , nan]